## Correction cas pratique 3

On commence par les premiers traitements : 

In [19]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# import du modèle
from sklearn.linear_model import LogisticRegression 
# import ACP
from sklearn.decomposition import PCA

from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000,-1)
X_train = sc.fit_transform(X_train)
X_test = X_test.reshape(10000,-1)
X_test = sc.fit_transform(X_test)

In [20]:
X_train.shape

(60000, 784)

La régression logistique sur l'ensemble des données était faite ainsi ( on sort le temps d'exécution avec `%%time`): 

In [21]:
%%time
# instanciation du modèle
reglog = LogisticRegression(solver='lbfgs', max_iter = 1000)
# entrainement du modèle 
reglog.fit(X_train, y_train)
# prédictions
y_pred = reglog.predict(X_test) # de tout le set de test


Wall time: 2min 9s


La précision du modèle est de : 

In [22]:
reglog.score(X_test, y_test)

0.9208

Pour utiliser la logique ACP, on n'a qu'à transformer X_train et X_test : 

In [23]:
pca = PCA(0.70)  # 70% de la variance expliquée
pca.fit(X_train) # On fitte sur X_train et on transforme x_train et X_test
projected_x_train = pca.transform(X_train) # pca.fit_transform = pca.fit() + pca.transform()
projected_x_test = pca.transform(X_test)
pca.n_components_

98

Pour 70% de la variance, on a 98 composantes. C'est bien moins que les 784 après le reshape

Le modèle qui tourne sur les X transformés ainsi donne : 

In [24]:
%%time
# On remet le PCA pour comparer de manière égale
pca = PCA(0.70)  # 70% de la variance expliquée
pca.fit(X_train) # On fitte sur X_train et on transforme x_train et X_test
projected_x_train = pca.transform(X_train) # pca.fit_transform = pca.fit() + pca.transform()
projected_x_test = pca.transform(X_test)

# instanciation du modèle
reglog = LogisticRegression(solver='lbfgs', max_iter = 1000)
# entrainement du modèle 
reglog.fit(projected_x_train, y_train)
# prédictions
y_pred = reglog.predict(projected_x_test) # de tout le set de test

Wall time: 48.4 s


On gagne beaucoup de temps! La précision est de :

In [25]:
reglog.score(projected_x_test, y_test)

0.9204

On peut tester avec moins de variance expliquée : 

In [26]:
%%time
# On remet le PCA pour comparer de manière égale
pca = PCA(0.65)  # 65% de la variance expliquée
pca.fit(X_train) # On fitte sur X_train et on transforme x_train et X_test
projected_x_train = pca.transform(X_train) # pca.fit_transform = pca.fit() + pca.transform()
projected_x_test = pca.transform(X_test)

# instanciation du modèle
reglog = LogisticRegression(solver='lbfgs', max_iter = 1000)
# entrainement du modèle 
reglog.fit(projected_x_train, y_train)
# prédictions
y_pred = reglog.predict(projected_x_test) # de tout le set de test

Wall time: 37.4 s


In [27]:
reglog.score(projected_x_test, y_test)

0.9176

Enfin, on peut tester en gardant une plus grande partie de la variance : 


In [28]:
pca = PCA(0.90)  # 90% de la variance expliquée
pca.fit(X_train) # On fitte sur X_train et on transforme x_train et X_test
projected_x_train = pca.transform(X_train) # pca.fit_transform = pca.fit() + pca.transform()
projected_x_test = pca.transform(X_test)
pca.n_components_

236

Ce qui donne :

In [29]:
%%time
# On remet le PCA pour comparer de manière égale
pca = PCA(0.90)  # 90% de la variance expliquée
pca.fit(X_train) # On fitte sur X_train et on transforme x_train et X_test
projected_x_train = pca.transform(X_train) # pca.fit_transform = pca.fit() + pca.transform()
projected_x_test = pca.transform(X_test)

# instanciation du modèle
reglog = LogisticRegression(solver='lbfgs', max_iter = 1000)
# entrainement du modèle 
reglog.fit(projected_x_train, y_train)
# prédictions
y_pred = reglog.predict(projected_x_test) # de tout le set de test

Wall time: 1min 2s


In [30]:
reglog.score(projected_x_test, y_test)

0.9273

A noter que si on prend une plus grosse variance on a...

In [31]:
# On remet le PCA pour comparer de manière égale
pca = PCA(0.98)  # 98% de la variance expliquée
pca.fit(X_train) # On fitte sur X_train et on transforme x_train et X_test
projected_x_train = pca.transform(X_train) # pca.fit_transform = pca.fit() + pca.transform()
projected_x_test = pca.transform(X_test)

# instanciation du modèle
reglog = LogisticRegression(solver='lbfgs', max_iter = 1000)
# entrainement du modèle 
reglog.fit(projected_x_train, y_train)
# prédictions
y_pred = reglog.predict(projected_x_test) # de tout le set de test

reglog.score(projected_x_test, y_test)

0.9258

... un moins bon résultat! C'est l'overfitting qui a mieux été contrôlé par l'ACP avec une variance expliquée plus faible.